## Multi-Blockchain Wallet in Python

In [1]:
# Import dependencies
import subprocess
import json
import os
import pprint

# Import necessary functions from bit and web3
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from dotenv import load_dotenv
from eth_account import Account
from web3 import Web3
from web3.gas_strategies.time_based import medium_gas_price_strategy

# Import constants from constants.py
from constants import *

## Nodes runing with POW

In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

## Load and set environment variables

In [3]:
load_dotenv()

True

## Load mnemonic

In [4]:
mnemonic = os.getenv("mnemonic")

## Create a function called `derive_wallets`

In [5]:
def derive_wallets(coin):
    """Use the subprocess library to create a shell command that calls the ./derive script from Python"""
    command = 'php ./derive -g --mnemonic="'+ mnemonic +'" --coin=' + coin + ' --numderive=3 --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

## Create a dictionary object called coins to store the output from `derive_wallets`.

In [6]:
coins = {
    ETH: derive_wallets(ETH),
    BTCTEST: derive_wallets(BTCTEST)
}

In [7]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(coins)

{ 'btc-test': [ { 'address': 'mgFom6U3NRxQNPBKX3nJCFxNpJ9xyqr7i1',
                  'index': 0,
                  'path': "m/44'/1'/0'/0/0",
                  'privkey': 'cPQecWpPMHz8prFQJcgJXkNtYY6q9EicmvnGGWSdUckvQWCSQxEY',
                  'pubkey': '03fe3db95714b7211f2c1fc33b09b9608f5505867b555a30f4857e7a785d7c0b2d',
                  'pubkeyhash': '081ac341a13704ad0e74f460e21b3dc82da1e0e3',
                  'xprv': 'tprv8jR2T3HCdScVrzC7iR4CNwdifnYWYsvUgQ3n8h2zkQ13uHMWLVSEgcyq5LTrKQ6wh2TZ5THthxmTdQiVvkMFGgj6zHe6SAhHLDe7W55Tx9m',
                  'xpub': 'tpubDG74bTKSmpJAkTDuc4innMHqEp4SiD7PFheZRD5JAfoSjmcGxtFps7bhFWsTV71Q4wFx66PkdE1FH4t8fcoEdcjCwKevS6EmxjY2X7EkDMp'},
                { 'address': 'msVBoDtk9UrSUjPj2n2RhdkeyxjcKiVUCD',
                  'index': 1,
                  'path': "m/44'/1'/0'/0/1",
                  'privkey': 'cMef4N1BTAaR2ZLxFFHEMx93SjF46vG3cZLy8hay4CqsvGNCCFPg',
                  'pubkey': '020831908859d896207d2e4a36f280649e10cf60b4baf97b674f78f12e47

In [8]:
eth_privkey = coins[ETH][0]['privkey']
btc_privkey = coins[BTCTEST][0]['privkey']

print(eth_privkey)
print(btc_privkey)

0x9be0a3547dea288bf6c6503b836d7aae36f35592b0057b10d627c6a540786d87
cPQecWpPMHz8prFQJcgJXkNtYY6q9EicmvnGGWSdUckvQWCSQxEY


## Create a function called `priv_key_to_account` that will convert the privkey string in a child key to an account object that bit or web3.py can use to transact

In [9]:
def priv_key_to_account(coin, priv_key):
    """Create a function called `priv_key_to_account` that converts privkey strings to account objects."""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [10]:
eth_acc = priv_key_to_account(ETH,eth_privkey)
btc_acc = priv_key_to_account(BTCTEST,btc_privkey) 

print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mgFom6U3NRxQNPBKX3nJCFxNpJ9xyqr7i1>


## Create a function called `create_tx` that will create the raw, unsigned transaction that contains all metadata needed to transact

In [11]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": to, "value": amount}
        )
        return {
            'to': to,
            'from': account.address,
            'value': amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

## Create a function called `send_tx` that will call create_tx, signs and sends the transaction

In [12]:
def send_tx(coin, account, to, amount):
    raw_tx = create_tx(coin, account, to, amount)
    signed_tx = account.sign_transaction(raw_tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

## Calling the functions to create, sign and execute transactions (BTCTest and ETH)

## Bitcoin Testnet transaction

In [12]:
# Create BTC transaction
create_tx(BTCTEST, btc_acc, "msVBoDtk9UrSUjPj2n2RhdkeyxjcKiVUCD", 0.001)

'{"unspents":[{"amount":333275,"confirmations":1,"script":"76a914081ac341a13704ad0e74f460e21b3dc82da1e0e388ac","txid":"9816ef0877d6098049742e5c392ec0c03cfc1dddbf2934e4b7640077cc7a4d01","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["msVBoDtk9UrSUjPj2n2RhdkeyxjcKiVUCD",100000],["mgFom6U3NRxQNPBKX3nJCFxNpJ9xyqr7i1",210223]]}'

In [14]:
#send BTC transaction
send_tx(BTCTEST,btc_acc,'msVBoDtk9UrSUjPj2n2RhdkeyxjcKiVUCD',0.001)

## Local PoA Ethereum transaction(Due to a bug in web3.py, you will need to send a transaction or two with MyCrypto first, since the w3.eth.generateGasPrice() function does not work with an empty chain. You can use one of the ETH address privkey, or one of the node keystore files)

In [11]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [12]:
from web3 import Web3, HTTPProvider

## Connecting to HTTP with address pk

In [13]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

## Checking the Block Number

In [14]:
w3.eth.blockNumber

74

## Double check if  I am connected to blockchain. 

In [17]:
w3.isConnected()

True

## Check the Balance of the account with local mining blockchain

In [18]:
w3.eth.getBalance("0x7c63fb6a44327EDBC025907B1226e0Fdb04fF6b8")

904625697166532776746648320380374280103671755200316906559262354061821324312

In [19]:
create_tx(ETH, eth_acc,"0x43d7F41ff92104A960bd2365E29fF72Ff26Ba1A2", 2000)

{'to': '0x43d7F41ff92104A960bd2365E29fF72Ff26Ba1A2',
 'from': '0x7c63fb6a44327EDBC025907B1226e0Fdb04fF6b8',
 'value': 2000,
 'gasPrice': 1000000000,
 'gas': 21000,
 'nonce': 1}

In [20]:
send_tx(ETH,eth_acc,"0x43d7F41ff92104A960bd2365E29fF72Ff26Ba1A2", 2000)

0x2141c2f2fc0d0a9d463c6faba6966b2167a4b9700646cb852fdf32bfa9dd2e81


'0x2141c2f2fc0d0a9d463c6faba6966b2167a4b9700646cb852fdf32bfa9dd2e81'

## Confirmation of Transaction

In [22]:
w3.eth.getBalance("0x43d7F41ff92104A960bd2365E29fF72Ff26Ba1A2")

1000000000000001000